Prompt Engineering 

In [ ]:
# Naive Prompt

from openai import OpenAI
client = OpenAI()
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages= [ 
        {"role":"system", "content":"You are an AI Assistant"},
        {"role":"user", "content":"Can I have 10 product names for a pair of shoes that can fit any foot size?"}
    ],
    temperature=0.8,
)

print(response.choices[0].message.content)

Sure! Here are ten product name ideas for a pair of shoes that can fit any foot size:

1. **FlexiFit Sneakers**
2. **Universal Sole**
3. **AdaptAll Shoes**
4. **OneSize Wonder**
5. **ShapeShift Sneakers**
6. **FitFlex Footwear**
7. **SizeSync Shoes**
8. **Malleable Motion**
9. **Dynamic Fit**
10. **VersaStep Shoes**

Feel free to mix and match or modify these names to better suit your branding!


In [5]:
# Structured Prompt

prompt = """
## Role
You are a creative product naming assistant.

## Task
Generate 10 unique and catchy product names for a pair of shoes that can fit any foot size.

## Instructions
- Output must be in valid JSON.
- Return only an array of product name strings (no explanations, no numbering).
- Names should be short, memorable, and marketable.
- Avoid duplicates.

## Output
[
  "Name1",
  "Name2"
]
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an AI assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.8,
)

print(response.choices[0].message.content)


[
  "FlexiFit",
  "SizeSway",
  "AdaptStep",
  "Universal Sole",
  "FitMates",
  "OneSize",
  "ElasticEase",
  "Freedom Walk",
  "SculptStep",
  "EverFit"
]
